# Data Loading and initial Filtering

## Load trackdata from MySQL database via SQL-Statement and saves waypoints to .parquet file for further use. 

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import json
import os
import pymysql
import pandas as pd
import pyarrow.parquet as pq  
import pyarrow as pa

with open('../Credentials/awm_database_credentials.json') as data_file:
    data = json.load(data_file)
    
HOST = data['host']
PORT = int(data['port']) 
USER = data['user']
PASSWORD = data['password']
DATABASE = data['database']

engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

query = """
SELECT 
    wp.id_tracking, wp.id, wp.time, wp.type, wp.sequence, wp.comment, 
    wp.speed, wp.heading, wp.duration, wp.block_type, wp.log, 
    wp.latitude, wp.longitude, wp.altitude, wp.meta_tag, wp.meta_value, t.length AS tracking_length, t.duration AS tracking_duration
FROM waypoint wp
JOIN tracking t ON wp.id_tracking = t.id
WHERE 
    t.duration BETWEEN 18000000000 AND 360000000000  -- Between 0.5 Hour and 10 Hours
    AND t.length BETWEEN 50 AND 850  -- Between 50km and 850km
    AND EXISTS (
        SELECT 1 FROM waypoint w 
        WHERE w.id_tracking = t.id
        HAVING COUNT(*) > 10  -- Ensure at least 10 waypoints exist
    )
    AND (
        (SELECT MAX(latitude) FROM waypoint WHERE id_tracking = t.id) - 
        (SELECT MIN(latitude) FROM waypoint WHERE id_tracking = t.id)
    ) > 0.0005  -- At least ~50m in latitude
    AND (
        (SELECT MAX(longitude) FROM waypoint WHERE id_tracking = t.id) - 
        (SELECT MIN(longitude) FROM waypoint WHERE id_tracking = t.id)
    ) > 0.0005  -- At least ~50m in longitude;
"""

parquet_file = "filtered_data_full_1.parquet"

conn = pymysql.connect(
    host=HOST,
    port=PORT,
    user=USER,
    password=PASSWORD,
    database=DATABASE,
    cursorclass=pymysql.cursors.SSCursor  
)

chunk_size = 1000000

try:
    writer = None

    for chunk in pd.read_sql(query, conn, chunksize=chunk_size):
        table = pa.Table.from_pandas(chunk)

        if writer is None:
            writer = pq.ParquetWriter(parquet_file, table.schema, compression="snappy")

        writer.write_table(table)
        print(f"Processed {len(chunk)} rows...")

    print("Data saved")

except Exception as e:
    print(f"Error: {e}")

finally:
    if writer:
        writer.close()
    conn.close()


/tmp/ipykernel_24210/3375776259.py:64: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  for chunk in pd.read_sql(query, conn, chunksize=chunk_size):


Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 1000000 rows...
Processed 10

## Determine the amount of trackings filtered

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import json
import os
import pymysql
import pandas as pd
import pyarrow.parquet as pq  
import pyarrow as pa

with open('../Credentials/awm_database_credentials.json') as data_file:
    data = json.load(data_file)
    
HOST = data['host']
PORT = int(data['port']) 
USER = data['user']
PASSWORD = data['password']
DATABASE = data['database']

engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")
conn = pymysql.connect(
    host=HOST,
    port=PORT,
    user=USER,
    password=PASSWORD,
    database=DATABASE,
    cursorclass=pymysql.cursors.SSCursor  
)

steps = [
    ("All trackings", "SELECT COUNT(*) FROM tracking"),
    
    ("Duration filter", 
     "SELECT COUNT(*) FROM tracking t WHERE t.duration BETWEEN 18000000000 AND 360000000000"),
    
    ("Length filter", 
     "SELECT COUNT(*) FROM tracking t WHERE t.duration BETWEEN 18000000000 AND 360000000000 AND t.length BETWEEN 50 AND 850"),
    
    ("Minimal Waypoints filter", 
     "SELECT COUNT(*) FROM tracking t WHERE t.duration BETWEEN 18000000000 AND 360000000000 AND t.length BETWEEN 50 AND 850 AND (SELECT COUNT(*) FROM waypoint w WHERE w.id_tracking = t.id) > 10"),
    
    ("Min Lat span filter", 
     "SELECT COUNT(*) FROM tracking t WHERE t.duration BETWEEN 18000000000 AND 360000000000 AND t.length BETWEEN 50 AND 850 AND (SELECT COUNT(*) FROM waypoint w WHERE w.id_tracking = t.id) > 10 AND ((SELECT MAX(latitude) FROM waypoint WHERE id_tracking = t.id) - (SELECT MIN(latitude) FROM waypoint WHERE id_tracking = t.id)) > 0.0005"),
    
    ("Min Long span filter", 
     "SELECT COUNT(*) FROM tracking t WHERE t.duration BETWEEN 18000000000 AND 360000000000 AND t.length BETWEEN 50 AND 850 AND (SELECT COUNT(*) FROM waypoint w WHERE w.id_tracking = t.id) > 10 AND ((SELECT MAX(latitude) FROM waypoint WHERE id_tracking = t.id) - (SELECT MIN(latitude) FROM waypoint WHERE id_tracking = t.id)) > 0.0005 AND ((SELECT MAX(longitude) FROM waypoint WHERE id_tracking = t.id) - (SELECT MIN(longitude) FROM waypoint WHERE id_tracking = t.id)) > 0.0005")
]


for name, sql in steps:
    result = pd.read_sql(sql, conn)
    print(f"{name}: {result.iloc[0,0]} trackings")


/tmp/ipykernel_5542/1591877189.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(sql, conn)


All trackings: 101353 trackings
Duration filter: 66477 trackings
Length filter: 45668 trackings
Minimal Waypoints filter: 45599 trackings


DatabaseError: Execution failed on sql: SELECT COUNT(*) FROM tracking t WHERE t.duration BETWEEN 18000000000 AND 360000000000 AND t.length BETWEEN 50 AND 850 AND (SELECT COUNT(*) FROM waypoint w WHERE w.id_tracking = t.id) > 10 AND ((SELECT MAX(latitude) FROM waypoint WHERE id_tracking = t.id) - (SELECT MIN(latitude) FROM waypoint WHERE id_tracking = t.id)) > 0.0005
(2013, 'Lost connection to MySQL server during query ([Errno 104] Connection reset by peer)')
unable to rollback